##### Copyright 2022 The Cirq Developers

In [ ]:
import sys
import os
import random
import numpy as np
import tensorflow as tf
from datetime import datetime, timedelta
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

# Ustawienia seedów
seed1_datetime = datetime.strptime("2024-11-26 20:15:08", "%Y-%m-%d %H:%M:%S")
seed1 = int(seed1_datetime.timestamp())
np.random.seed(seed1)
tf.random.set_seed(seed1)

# Definiowanie target pattern
target_pattern = [1, 2, 3, 4, 5, 20, 21, 28, 32, 37]

# Parametry danych
rows_10_50 = 500_000
cols_10_50 = len(target_pattern)  # Liczba kolumn = długość target_pattern
number_range_50 = range(1, 51)
excluded_values_50 = target_pattern

# Funkcja generująca dane
def generate_2d_array(rows, cols, number_range, excluded_values=None):
    if excluded_values is None:
        excluded_values = []
    available_numbers = [num for num in number_range if num not in excluded_values]
    if len(available_numbers) < cols:
        raise ValueError("Za mało dostępnych liczb.")
    data = [sorted(random.sample(available_numbers, cols)) for _ in range(rows)]
    return np.array(data)

# Generowanie tablicy
data = generate_2d_array(rows_10_50, cols_10_50, number_range_50, excluded_values_50)
print(data)

# Skalowanie danych
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(data)

# Parametry modelu
input_dim = cols_10_50
latent_dim = 500_000
num_mixtures = 1_000

# Funkcja straty MDN
def mdn_loss(num_mixtures, output_dim):
    def loss(y_true, outputs):
        alphas, mus, sigmas = tf.split(outputs, [
            num_mixtures, num_mixtures * output_dim, num_mixtures * output_dim
        ],
                                       axis=-1)
        mus = tf.reshape(mus, [-1, num_mixtures, output_dim])
        sigmas = tf.reshape(sigmas, [-1, num_mixtures, output_dim])
        y_true = tf.expand_dims(y_true, axis=1)
        gaussians = tf.exp(
            -0.5 * tf.reduce_sum(tf.square(
                (y_true - mus) / sigmas), axis=-1)) / (
                    tf.reduce_prod(sigmas, axis=-1) * tf.sqrt(2.0 * np.pi))
        weighted_gaussians = alphas * gaussians
        nll = -tf.math.log(tf.reduce_sum(weighted_gaussians, axis=-1) + 1e-8)
        return tf.reduce_mean(nll)
    return loss

# Tworzenie modelu MDN + VAE
def create_mdn_vae_model(input_dim, latent_dim, num_mixtures):
    inputs = Input(shape=(input_dim,))
    h = Dense(64, activation='relu')(inputs)
    h = Dense(32, activation='relu')(h)
    z_mean = Dense(latent_dim, name='z_mean')(h)
    z_log_var = Dense(latent_dim, name='z_log_var')(h)

    def sampling(args):
        z_mean, z_log_var = args
        epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    h_decoder = Dense(32, activation='relu')(z)
    h_decoder = Dense(64, activation='relu')(h_decoder)

    alphas = Dense(num_mixtures, activation='softmax', name='alphas')(h_decoder)
    mus = Dense(num_mixtures * input_dim, name='mus')(h_decoder)
    sigmas = Dense(num_mixtures * input_dim, activation='softplus', name='sigmas')(h_decoder)

    outputs = Lambda(lambda x: tf.concat(x, axis=-1), name='mdn_output')([alphas, mus, sigmas])

    mdn_vae = Model(inputs, outputs, name='mdn_vae')
    return mdn_vae

# Tworzenie i kompilacja modelu
model = create_mdn_vae_model(input_dim, latent_dim, num_mixtures)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=mdn_loss(num_mixtures, input_dim))

# Trening modelu
model.fit(X_scaled, X_scaled, epochs=500_000, batch_size=500_000, validation_split=0.1, verbose=1)

# Funkcja oceny próbki
def evaluate_sample(model, scaler, sample, target_pattern):
    output = model.predict(sample, verbose=0)
    alphas, mus, sigmas = tf.split(output, [
        num_mixtures, num_mixtures * input_dim, num_mixtures * input_dim
    ], axis=-1)
    mus = tf.reshape(mus, [-1, num_mixtures, input_dim])
    prediction = scaler.inverse_transform(tf.reshape(mus[0, 0, :], (1, -1)).numpy())
    generated_numbers = sorted(np.round(np.clip(prediction[0], 1, 50)).astype(int))
    score = np.sum(np.abs(np.array(generated_numbers) - np.array(target_pattern)))
    return -score

# Funkcja MCMC
def metropolis_hastings(model, scaler, target_pattern, num_samples=500_000, input_dim=10):
    current_sample = np.random.normal(size=(1, input_dim))
    accepted_samples = []
    current_score = evaluate_sample(model, scaler, current_sample, target_pattern)

    with tqdm(total=num_samples, desc="MCMC Progress", unit="sample") as pbar:
        for _ in range(num_samples):
            proposed_sample = current_sample + np.random.normal(0, 0.1, size=current_sample.shape)
            proposed_score = evaluate_sample(model, scaler, proposed_sample, target_pattern)
            acceptance_prob = min(1, np.exp(current_score - proposed_score))
            if np.random.rand() < acceptance_prob:
                current_sample = proposed_sample
                current_score = proposed_score
                accepted_samples.append(current_sample)
            pbar.update(1)
    return np.array(accepted_samples)

# Wyszukiwanie wzorca
def find_pattern_with_mcmc(model, scaler, target_pattern, max_samples=500_000):
    print(f"Rozpoczynam wyszukiwanie wzorca {target_pattern} przy użyciu MCMC...")
    accepted_samples = metropolis_hastings(model, scaler, target_pattern, num_samples=max_samples)
    for sample in accepted_samples:
        generated_numbers = sorted(np.round(np.clip(sample[0], 1, 50)).astype(int))
        if generated_numbers == target_pattern:
            print(f"\nZnaleziono wzorzec: {generated_numbers}")
            break
    else:
        print("\nNie znaleziono wzorca w wygenerowanych próbkach.")

find_pattern_with_mcmc(model, scaler, target_pattern)

# Hello Qubit

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/start/start"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/main/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/main/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    import cirq

    print("installed cirq.")

In [ ]:
# Pick a qubit.
qubit = cirq.GridQubit(0, 0)

# Create a circuit that applies a square root of NOT gate, then measures the qubit.
circuit = cirq.Circuit(cirq.X(qubit) ** 0.5, cirq.measure(qubit, key='m'))
print("Circuit:")
print(circuit)

# Simulate the circuit several times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print("Results:")
print(result)

# Congratulations
You've just run your first Cirq program.

To learn about running a circuit on a virtual machine that mimics existing quantum hardware, see [Quantum Virtual Machine](../simulate/quantum_virtual_machine.ipynb).

If you would like to learn more about quantum computing, check out our [education page](https://quantumai.google/resources). The Full API reference for Cirq can be found [here](/reference/python/cirq). If you are looking for vendor specific information that can be found on our vendor sub-pages:


  [Alpine Quantum Technologies](../hardware/aqt/getting_started.ipynb)
  
  [Pasqal](../hardware/pasqal/getting_started.ipynb)
  
  [IonQ](../hardware/ionq/getting_started.ipynb)
  
  [Azure](../hardware/azure-quantum/getting_started_honeywell.ipynb)
  
  [Rigetti](../hardware/rigetti/getting_started.ipynb)